<a href="https://colab.research.google.com/github/skaurl/NH-bigdata/blob/main/classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 55.0MB 103kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 12.8MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588515 sha256=27b42950c388e2941fd5d5bdd8be5a0332ed6410e178bae30d7fc0af91887338
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 14.0MB/s 
     |████████████████████████████████| 757kB 13.9MB/s 
     |████████████████████████████████| 3.0MB 31.9MB/s 
     |████████████████████████████████| 890kB 52.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=cae33e268bc1a6999260e8cdc306f11ee97dd0e603e53304f84365cb2ea73c06
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
device = torch.device("cuda:0")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


# Preprocessing

In [6]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

news_dataset_path = r"/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/1.Data/open/news_train.csv"

news_dataset = pd.read_csv(news_dataset_path)

news_dataset = news_dataset[news_dataset["content"]!="데이터에서 찾은 AI 속보"]
news_dataset = news_dataset[news_dataset["content"]!="씨에스윈드, 193억 원 규모 윈드타워 공급계약 체결"]
news_dataset = news_dataset[news_dataset["content"]!="씨에스윈드, 1분기 영업이익 158억 원전년비 16.1%"]

news_dataset["content"] = news_dataset["title"] + " " + news_dataset["content"]

del news_dataset["n_id"]
del news_dataset["date"]
del news_dataset["title"]
del news_dataset["ord"]

news_dataset = news_dataset.drop_duplicates()

news_dataset["content"] = news_dataset["content"].apply(lambda x : ' '.join(x.strip().split()))

train, test = train_test_split(news_dataset, test_size=0.1, random_state=42)

train.to_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/1.Data/news_train_train.txt", sep='\t', index=False)
test.to_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/1.Data/news_train_test.txt", sep='\t', index=False)

In [7]:
dataset_train = nlp.data.TSVDataset("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/1.Data/news_train_train.txt", field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/1.Data/news_train_test.txt", field_indices=[0,1], num_discard_samples=1)

In [8]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
max_len = 128
batch_size = 64
warmup_ratio = 0.01
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [12]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

# Classification

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [15]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [16]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [17]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [18]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [19]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.7145305871963501 train acc 0.4375
epoch 1 batch id 201 loss 0.10184042155742645 train acc 0.921875
epoch 1 batch id 401 loss 0.03117986023426056 train acc 0.9464230049875312
epoch 1 batch id 601 loss 0.07595422118902206 train acc 0.9580906821963394
epoch 1 batch id 801 loss 0.1335894614458084 train acc 0.9649071473158551

epoch 1 train acc 0.969075859654435


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9903417448060304


epoch 2 batch id 1 loss 0.01774946041405201 train acc 1.0
epoch 2 batch id 201 loss 0.037871070206165314 train acc 0.9896610696517413
epoch 2 batch id 401 loss 0.008383787237107754 train acc 0.9909211346633416
epoch 2 batch id 601 loss 0.0782151073217392 train acc 0.9920445091514143
epoch 2 batch id 801 loss 0.0650489404797554 train acc 0.99270443196005

epoch 2 train acc 0.9933550651955868


...


epoch 3 test acc 0.993384009009009


epoch 4 batch id 1 loss 0.0004114733310416341 train acc 1.0
epoch 4 batch id 201 loss 0.00020492012845352292 train acc 0.9992226368159204
epoch 4 batch id 401 loss 0.00018793960043694824 train acc 0.9991038029925187
epoch 4 batch id 601 loss 0.0004016011953353882 train acc 0.9992720465890182
epoch 4 batch id 801 loss 0.00012136494478909299 train acc 0.9993172596754057

epoch 4 train acc 0.9993261033099298



epoch 4 test acc 0.9943262778084208


epoch 5 batch id 1 loss 0.000324369320878759 train acc 1.0
epoch 5 batch id 201 loss 0.00014836004993412644 train acc 0.9999222636815921
epoch 5 batch id 401 loss 0.00010512120206840336 train acc 0.9998441396508728
epoch 5 batch id 601 loss 0.0001154237543232739 train acc 0.9998700083194676
epoch 5 batch id 801 loss 8.862611139193177e-05 train acc 0.9999024656679151

epoch 5 train acc 0.9998746238716149



epoch 5 test acc 0.9943693693693694


In [ ]:
torch.save(model.state_dict(), "/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/5.Model/model_state_dict.pt")

# Scoring

데이터 불러오기

In [20]:
import pandas as pd

news_dataset_path = r"/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/1.Data/open/news_test.csv"

news_dataset = pd.read_csv(news_dataset_path)

시간 측정 시작

In [21]:
import time
start = time.time()

Library 불러오기

In [22]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

Model 불러오기

In [23]:
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

model.load_state_dict(torch.load("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/5.Model/model_state_dict.pt"))

using cached model
using cached model


<All keys matched successfully>

형태소 분석 + 전처리

In [24]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

news_dataset["content"] = news_dataset["title"] + " " + news_dataset["content"]

del news_dataset["n_id"]
del news_dataset["date"]
del news_dataset["title"]
del news_dataset["ord"]

news_dataset["content"] = news_dataset["content"].apply(lambda x : ' '.join(x.strip().split()))

news_dataset["info"] = 0

news_dataset[["content","info"]].to_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/1.Data/news_test.txt", sep='\t', index=False)

dataset_test = nlp.data.TSVDataset("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/코난/1.Data/news_test.txt", field_indices=[0,1], num_discard_samples=1)

data_test = BERTDataset(dataset_test, 0, 1, tok, 128, True, False)

test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=64, num_workers=5)

using cached model


예측

In [25]:
model.eval()

Predict = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _, predict = torch.max(out,1)
    Predict.extend(predict.tolist())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [26]:
news_dataset["info"] = Predict

print(news_dataset.groupby("info").size())

news_dataset = news_dataset[["id","info"]]

info
0    88161
1    54404
dtype: int64


In [27]:
print(time.time() - start)

408.93971490859985


제출

In [ ]:
news_dataset.to_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/result/sample_submission.csv", index=False)